# Income Statement Sensitivity Analysis with Python

we are going to perform an income statement sensitivity analysis using Python. 
We will first retrieve the income statement for a company.
Then, we will check how an increase of sales impact net income by projecting the whole income statement.
As in my previous entries in Python for Finance, all what we need is Python.

# Why Sensitivity Analysis in Finance?

Sensitivity analysis is used to project values based on changes in an independent variable. This method is particularly useful to answer what if questions.

We can apply sensitivity analysis in finance in order to analyse what if scenarios in any metrics or company financials that we are interested in. For example, we can analyse what will be the effect on net income if sales drop by 10%. Or we could simply have a range of values for sales and based on them, simulate potential effects on different financial ratios or company financial statements.

In this post, we will only apply the sensitivity analysis into income statement projections. Note that sensitivity analysis in finance has many other use cases. For example, we could predict the price of a stock by changing different variables affecting stock prices such as company earnings, debt ratio, etc.

Lets have a look at how to perform such a sensitivity analysis in a company income statement using Python.

# Income Statement with Python

Before moving with Python, below are the steps that we are going to follow. For this example, we will be looking into Reliance income statement.

1.Retrieve latest yearly Reliance Income Statement using the free financialmodelingprep API.

2.Calculate each income statement item as a percentage of revenue. This will be used to calculate the projection of the income statement based on different sensitivity values (i.e. increase in sales of 10%).


3.Project a good outcome income statement. We will calculate and simulate an increase of sales and project the rest of the income statement items based on this change.


4.Project a bad outcome (i.e. negative sales) income statement.


5.Represent the actual and projected income statements in a Pandas DataFrame.


To start with the Python code, we will perform an http request to retrieve Reliance income statement for the last year. You can get your free api key in financialmodelingprep and pass it as an argument in the url:

In [2]:
import pandas as pd
import requests
import numpy as np

In [3]:
demo='4591b1cb907ba0fb42cd870c9a3b4422'
stock = 'RELIANCE.NS'

In [4]:
IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{stock}?apikey={demo}').json()

IS2019 = IS[0]

Then, we define our sensitivity values. In this case, we want to see the effect on the income statement if revenues grow by 3%. That will be our good outcome scenario. In addition, we will also check the effect of a 10% reduction of sales.

In [10]:
growth=1.05
negative_growth=0.93

#Create the Dictionaries
sensititvity =  {}
sensititvity['last_year'] = {}
sensititvity['projection'] = {}
sensititvity['projection_bad'] = {}

We will add the three income statements, that is the actual and the two projected ones, into a Python dictionary. The dictionary will contain three nested dictionaries with each of the three income statements.

Next, we parse the API returned data and add each of the variables into the last_year dictionary by parsing the relevant keys.

You can see here, the structure and the key value pairs of the returned dictionary. Below is the code to extract the income statement line items and add them to our dictionary.

To keep it short, I will only extract a few of the income statement line items as per below code:



In [11]:
#latest available data
sensititvity['last_year']['revenue'] = IS2019['revenue']
sensititvity['last_year']['COGS'] = IS2019['costOfRevenue']
sensititvity['last_year']['grossProfit'] = IS2019['grossProfit']
sensititvity['last_year']['opinc'] = IS2019['operatingIncome']
sensititvity['last_year']['intexp'] = IS2019['interestExpense']
sensititvity['last_year']['netIncome'] = IS2019['netIncome']

Having now the latest income statement for Reliance into the last_year dictionary, we can move on and calculate each of the line items as a percentage of sales. This percentage will be our key to determine each of the line items of the projected income statement other than sales.

In [12]:
#item as a percentage of sales
sensititvity['last_year']['COGSpersales'] = sensititvity['last_year']['COGS'] /sensititvity['last_year']['revenue']
sensititvity['last_year']['grossProfitpersales'] = sensititvity['last_year']['grossProfit'] /sensititvity['last_year']['revenue']
sensititvity['last_year']['opincpersales'] = sensititvity['last_year']['opinc']/sensititvity['last_year']['revenue']
sensititvity['last_year']['intexppersales'] = sensititvity['last_year']['intexp']/sensititvity['last_year']['revenue']
sensititvity['last_year']['netIncomepersales']  = sensititvity['last_year']['netIncome']/sensititvity['last_year']['revenue']

#  Projecting the Income Statement

the good projection, note that we calculate the increase of revenue by multiplying revenue by the growth rate. Then, the rest of the income statement items are calculated using the percentage of sales allocation that we computed above:

In [24]:
#good projection
sensititvity['projection']['revenue'] = sensititvity['last_year']['revenue'] * growth
sensititvity['projection']['COGS'] = sensititvity['last_year']['COGSpersales'] * sensititvity['projection']['revenue'] 
sensititvity['projection']['grossProfit'] = sensititvity['last_year']['grossProfitpersales'] * sensititvity['projection']['revenue'] 
sensititvity['projection']['opinc'] = sensititvity['last_year']['opincpersales'] * sensititvity['projection']['revenue'] 
sensititvity['projection']['intexp'] = sensititvity['last_year']['intexppersales'] * sensititvity['projection']['revenue'] 
sensititvity['projection']['netIncome'] = sensititvity['last_year']['netIncomepersales'] * sensititvity['projection']['revenue'] 

In [25]:
#bad projection
sensititvity['projection_bad']['revenue'] = sensititvity['last_year']['revenue'] * negative_growth
sensititvity['projection_bad']['COGS'] = sensititvity['last_year']['COGSpersales'] * sensititvity['projection_bad']['revenue'] 
sensititvity['projection_bad']['grossProfit'] = sensititvity['last_year']['grossProfitpersales'] * sensititvity['projection_bad']['revenue'] 
sensititvity['projection_bad']['opinc'] = sensititvity['last_year']['opincpersales'] * sensititvity['projection_bad']['revenue'] 
sensititvity['projection_bad']['intexp'] = sensititvity['last_year']['intexppersales'] * sensititvity['projection_bad']['revenue'] 
sensititvity['projection_bad']['netIncome'] = sensititvity['last_year']['netIncomepersales'] * sensititvity['projection_bad']['revenue'] 

# Reliance sensitivity analysis – overview

In the last step, we are going to represent the data in a Pandas DataFrame following the income statement structure. To do this, we are going to transform our three dictionaries into a Pandas DataFrame and perform some basic data manipulations.

First, lets create the Pandas DataFrame using the method pd.DataFrame.from_dict(). Note that we pass our dictionary as an argument:

In [26]:
sensitivity_analysis = pd.DataFrame.from_dict(sensititvity,orient='columns')

In [27]:
#show in milions
sensitivity_analysis = sensitivity_analysis/1000000
sensitivity_analysis.reset_index(inplace=True)

sensitivity_analysis = sensitivity_analysis[~sensitivity_analysis["index"].str.contains('persales')]
sensitivity_analysis


,index,last_year,projection,projection_bad
0,revenue,5965540.0,6263817.0,5547952.2
1,COGS,4512470.0,4738093.5,4196597.1
2,grossProfit,1453070.0,1525723.5,1351355.1
3,opinc,662710.0,695845.5,616320.3
4,intexp,199350.0,209317.5,185395.5
5,netIncome,393540.0,413217.0,365992.2
